#**01. Instalar SQL para conectarse con la base de datos.**
-----------------------------------------------
Existen diferentes librerías para trabajar con bases de datos tipo SQL en Python.   \\
Entre las más conocidos tenemos a :

*   MySQL
*   SQL lite

En esta ocasión trabajaremos con **SQL lite 3** que viene precargada en Google colab




In [ ]:
import sqlite3

##01.1. **Establecemos conexión mediante sqlite3**
SQL lite nos permite establecer y cerrar conexión con una base de datos mediante los siguientes comandos:

In [ ]:
#Abrimos conexión
conexión = sqlite3.connect('Mi_nueva_base.db')
print('Conexión establecida')

#Cerramos conexión
conexión.close()
print('Conexión terminada')

Conexión establecida
Conexión terminada


##01.2. **Ejecutar comandos SQL en sqlite3**
Para usar comandos SQL como queries o preguntas, es necesario crear un objeto de clase cursor que pueda ejecutarlas. \\
  **`nombre_cursor=conexión.cursor()`** \\
Y para ejecutar comandos SQL en el entorno creado: \\
**`nombre_cursor.execute(comando_sql_predefinido)`**

In [ ]:
#Abrimos conexión
conexión = sqlite3.connect('Mi_nueva_base.db')
print('Conexión establecida')


#Creamos objeto cursor
cursor=conexión.cursor()

#comando que queremos ejecutar
comando_sql = '''
              CREATE TABLE IF NOT EXISTS nombre_tabla (
                identificador INTEGER PRIMARY KEY,
                nombre VARCHAR(20),
                apellidos VARCHAR(30),
                gender CHAR(1));
              '''
#Ejecutar comado SQL con nuestro cursor
cursor.execute(comando_sql)
print("El comando se ejecutó sin errores")


#Cerramos conexión
conexión.close()
print('Conexión terminada')

Conexión establecida
El comando se ejecutó sin errores
Conexión terminada


#**02. USANDO PANDAS CON SQL**
--------------------------------
 En SQL encontraremos bases de datos a las que podemos acceder y extraer datos para ser analizados mediante pandas. \\


```
import pandas as pd

pd.read_sql( querie_sql, nombre_de_conexión )
```
Del mismo modo podemos convertimos un dataframe a formato sql **y agregarlo a una base de datos a la que accedimos mediante la conexión sqlite** :


```
nombre_dataframe.to_sql( nuevo_nombre, nombre_de_conexión , if_exists='replace',  index=False')
```
Debido a que puede existir una tabla con el mismo nombre (si corres dos veces el código por ejemplo), ponemos que la nueva tabla reemplace a la anterior de existir y que no considere el índice.




##**02.1. Creo una base de datos SQL, en este caso VARIANTES_COVID_PERÚ**

Crear una base de datos es tan simple como establecer una conexión y asiganarle un nombre :    
**`sqlite3.connect(nombre_base_de_datos.db)`**
No cerraremos la conexión, ya que trabajaremos con ella de ahora en adelante.


In [ ]:
conexión_a_variantes_covid_perú = sqlite3.connect('VARIANTES_COVID_PERÚ.db')

##**02.2. AGREGANDO UN DATAFRAME A UNA BASE DE DATOS**
Para este ejemplo usaré la base de datos reportados hasta el 25 de setiembre por el minsa.

In [ ]:
'''
Primero importo pandas y numpy ,que hasta el momento
no lo había hecho
'''
import pandas as pd
import numpy as np

'''
Creo el dataframe genomas reportados, usando el archivo csv colgado en datos abiertos del minsa
'''
genomas_reportados = pd.read_csv('/content/pmGenoma_25Septiembre2021.csv', sep='|')
genomas_reportados.head(5)

,FECHA_CORTE,UUID,FECHA_MUESTRA,Edad,Sexo,Institucion,UBIGEO_PACIENTE,DEPARTAMENTO_PACIENTE,PROVINCIA_PACIENTE,DISTRITO_PACIENTE,DEPARTAMENTO_MUESTRA,PROVINCIA_MUESTRA,DISTRITO_MUESTRA,TIPO_MUESTRA,RESULTADO
0,20210925,NaN,20210209.0,93.0,FEMENINO,GOBIERNO REGIONAL,190115.0,PIURA,PIURA,VEINTISEIS DE OCTUBRE,PIURA,PIURA,PIURA,HISOPADO NASAL Y FARINGEO,Linaje B.1.1
1,20210925,NaN,20210222.0,31.0,FEMENINO,GOBIERNO REGIONAL,120101.0,LA LIBERTAD,TRUJILLO,TRUJILLO,LA LIBERTAD,TRUJILLO,TRUJILLO,HISOPADO NASAL Y FARINGEO,Linaje C.37
2,20210925,NaN,20210301.0,27.0,MASCULINO,GOBIERNO REGIONAL,120103.0,LA LIBERTAD,TRUJILLO,LAREDO,LA LIBERTAD,TRUJILLO,LAREDO,HISOPADO NASAL Y FARINGEO,Linaje C.37
3,20210925,NaN,20210307.0,34.0,MASCULINO,GOBIERNO REGIONAL,240102.0,,CALLAO,BELLAVISTA,CALLAO,PROV. CONST. DEL CALLAO,BELLAVISTA,HISOPADO NASAL Y FARINGEO,Linaje C.37
4,20210925,NaN,20210307.0,54.0,MASCULINO,GOBIERNO REGIONAL,240102.0,,CALLAO,BELLAVISTA,CALLAO,PROV. CONST. DEL CALLAO,BELLAVISTA,HISOPADO NASAL Y FARINGEO,Linaje C.37


In [ ]:
'''
REALIZARÉ MODIFICACIONES A LA COLUMNA FECHA DE MUESTRA Y FECHA DE CORTE PARA QUE
SUS VALORES SEAN REGISTRADOS COMO FORMATO DATETIME O FECHA
'''
#Eliminación de filas con valores vacíos en las fechas
genomas_reportados.dropna(subset=['FECHA_MUESTRA'],inplace=True)
genomas_reportados.dropna(subset=['FECHA_CORTE'],inplace=True)

#Conversión de los valores númericos a los formatos fecha
genomas_reportados['FECHA_MUESTRA'] = genomas_reportados.apply(lambda x : (pd.to_datetime( str(int(x['FECHA_MUESTRA'])), format="%Y%m%d")) , axis=1 )
genomas_reportados['FECHA_CORTE'] =   genomas_reportados.apply(lambda x : (pd.to_datetime( str(int(  x['FECHA_CORTE'])), format="%Y%m%d")) , axis=1 )

#Observación de los últimos 5 valores
genomas_reportados.tail(5)

,FECHA_CORTE,UUID,FECHA_MUESTRA,Edad,Sexo,Institucion,UBIGEO_PACIENTE,DEPARTAMENTO_PACIENTE,PROVINCIA_PACIENTE,DISTRITO_PACIENTE,DEPARTAMENTO_MUESTRA,PROVINCIA_MUESTRA,DISTRITO_MUESTRA,TIPO_MUESTRA,RESULTADO
6366,2021-09-25,37259371.0,2021-08-18,36.0,FEMENINO,GOBIERNO REGIONAL,40109.0,AREQUIPA,AREQUIPA,PAUCARPATA,AREQUIPA,AREQUIPA,JACOBO HUNTER,HISOPADO NASAL Y FARINGEO,Linaje B.1.617.2
6367,2021-09-25,37269802.0,2021-08-01,51.0,MASCULINO,GOBIERNO REGIONAL,240101.0,,CALLAO,CALLAO,CALLAO,PROV. CONST. DEL CALLAO,CALLAO,HISOPADO NASAL Y FARINGEO,Linaje AY.4
6368,2021-09-25,37274411.0,2021-01-03,42.0,MASCULINO,GOBIERNO REGIONAL,140106.0,LIMA,LIMA,COMAS,CALLAO,PROV. CONST. DEL CALLAO,CALLAO,HISOPADO NASAL Y FARINGEO,Linaje B.1.2
6369,2021-09-25,37283811.0,2021-06-10,24.0,FEMENINO,GOBIERNO REGIONAL,40107.0,AREQUIPA,AREQUIPA,MIRAFLORES,AREQUIPA,AREQUIPA,MIRAFLORES,HISOPADO NASAL Y FARINGEO,Linaje P.1
6370,2021-09-25,37287342.0,2021-04-09,26.0,MASCULINO,GOBIERNO REGIONAL,140805.0,LIMA,HUARAL,CHANCAY,LIMA,HUARAL,CHANCAY,HISOPADO NASAL Y FARINGEO,Linaje P.1


In [ ]:
genomas_reportados

,FECHA_CORTE,UUID,FECHA_MUESTRA,Edad,Sexo,Institucion,UBIGEO_PACIENTE,DEPARTAMENTO_PACIENTE,PROVINCIA_PACIENTE,DISTRITO_PACIENTE,DEPARTAMENTO_MUESTRA,PROVINCIA_MUESTRA,DISTRITO_MUESTRA,TIPO_MUESTRA,RESULTADO
0,2021-09-25,NaN,2021-02-09,93.0,FEMENINO,GOBIERNO REGIONAL,190115.0,PIURA,PIURA,VEINTISEIS DE OCTUBRE,PIURA,PIURA,PIURA,HISOPADO NASAL Y FARINGEO,Linaje B.1.1
1,2021-09-25,NaN,2021-02-22,31.0,FEMENINO,GOBIERNO REGIONAL,120101.0,LA LIBERTAD,TRUJILLO,TRUJILLO,LA LIBERTAD,TRUJILLO,TRUJILLO,HISOPADO NASAL Y FARINGEO,Linaje C.37
2,2021-09-25,NaN,2021-03-01,27.0,MASCULINO,GOBIERNO REGIONAL,120103.0,LA LIBERTAD,TRUJILLO,LAREDO,LA LIBERTAD,TRUJILLO,LAREDO,HISOPADO NASAL Y FARINGEO,Linaje C.37
3,2021-09-25,NaN,2021-03-07,34.0,MASCULINO,GOBIERNO REGIONAL,240102.0,,CALLAO,BELLAVISTA,CALLAO,PROV. CONST. DEL CALLAO,BELLAVISTA,HISOPADO NASAL Y FARINGEO,Linaje C.37
4,2021-09-25,NaN,2021-03-07,54.0,MASCULINO,GOBIERNO REGIONAL,240102.0,,CALLAO,BELLAVISTA,CALLAO,PROV. CONST. DEL CALLAO,BELLAVISTA,HISOPADO NASAL Y FARINGEO,Linaje C.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6366,2021-09-25,37259371.0,2021-08-18,36.0,FEMENINO,GOBIERNO REGIONAL,40109.0,AREQUIPA,AREQUIPA,PAUCARPATA,AREQUIPA,AREQUIPA,JACOBO HUNTER,HISOPADO NASAL Y FARINGEO,Linaje B.1.617.2
6367,2021-09-25,37269802.0,2021-08-01,51.0,MASCULINO,GOBIERNO REGIONAL,240101.0,,CALLAO,CALLAO,CALLAO,PROV. CONST. DEL CALLAO,CALLAO,HISOPADO NASAL Y FARINGEO,Linaje AY.4
6368,2021-09-25,37274411.0,2021-01-03,42.0,MASCULINO,GOBIERNO REGIONAL,140106.0,LIMA,LIMA,COMAS,CALLAO,PROV. CONST. DEL CALLAO,CALLAO,HISOPADO NASAL Y FARINGEO,Linaje B.1.2
6369,2021-09-25,37283811.0,2021-06-10,24.0,FEMENINO,GOBIERNO REGIONAL,40107.0,AREQUIPA,AREQUIPA,MIRAFLORES,AREQUIPA,AREQUIPA,MIRAFLORES,HISOPADO NASAL Y FARINGEO,Linaje P.1


In [ ]:
'''
Agregamos el dataframe genomas_reportados a la base de datos VARIANTES_COVID_PERÚ
'''
try :
  genomas_reportados.to_sql('variantes_reportadas_en_perú', conexión_a_variantes_covid_perú, if_exists='replace', index=False)
  print('Dataframe agregado a base de datos de forma exitosa')
except:
  print('El DataFrame ya ha sido agregado previamente.')

Dataframe agregado a base de datos de forma exitosa


##**02.3. EXTRAER UN DATAFRAME DE UNA BASE DE DATOS SQL**

Pandas tiene la opción de leer archivos o tablas sql , pero para esto necesita instrucciones tipo querie o pregunta sql :


```
ejemplo de querie = '''
                      SELECT nombre_de_columnas
                      FROM nombre_de_tabla
                      WHERE condiciones
                      '''
```



In [ ]:
'''
Para este ejemplo usaré una función, ya que al ejecutar una función,
si la función no es guardada dentro de un objeto, todo su contenido como
resultado es eliminado de la memoria
'''
'''
Para este ejemplo mediante el query selecionaré tres columnas de mi interés,
de la tabla variantes_reportadas_en_perú, y filtraré a solo los reportes donde
se registre a la provincia del paciente como BARRANCA.
'''
def obtener_dataframe():

  querie='''
          SELECT Edad,Sexo, DISTRITO_PACIENTE, RESULTADO
          FROM  variantes_reportadas_en_perú
          WHERE PROVINCIA_PACIENTE = 'BARRANCA';
          '''
  dataframe = pd.read_sql(querie, conexión_a_variantes_covid_perú)

  return dataframe

#Ejecuto mi función
obtener_dataframe()

,Edad,Sexo,DISTRITO_PACIENTE,RESULTADO
0,68.0,MASCULINO,SUPE,Linaje B.1.621
1,66.0,MASCULINO,BARRANCA,Linaje C.37
2,33.0,MASCULINO,BARRANCA,Linaje B.1.617.2
3,35.0,FEMENINO,BARRANCA,Linaje P.1
4,31.0,MASCULINO,BARRANCA,Linaje B.1.617.2
5,25.0,MASCULINO,BARRANCA,Linaje B.1.617.2
6,42.0,MASCULINO,BARRANCA,Linaje C.37
7,14.0,FEMENINO,BARRANCA,Linaje AY.12
8,37.0,FEMENINO,BARRANCA,Sublinaje C.37


In [ ]:
"""
MEDIANTE EL CURSOR TAMBIÉN SE PUEDE RECOGER LA BASE DE DATOS, SIN EMBARGO NO ES ÓPTIMO YA
QUE MEDIANTE cursor.fetchall() RECUPERAS LA MATRIZ DE RESULTADOS SIN ÍNDICES NI ENCABEZADOS.

"""
def mediante_sql():

  cursor=conexión_a_variantes_covid_perú.cursor()

  comando_sql = '''
                SELECT Edad,Sexo, DISTRITO_PACIENTE, RESULTADO
                FROM  variantes_reportadas_en_perú
                WHERE PROVINCIA_PACIENTE = 'BARRANCA';
                '''
  # Ejecutar comado SQL con nuestro cursor
  cursor.execute(comando_sql)

  # Mediante cursor.fetchall() creamos una matriz con los resultados
  # Podemos guardar esos resultados en un database, pero no tendrán
  # los cabezales
  print(cursor)
  c=pd.DataFrame(cursor.fetchall())

  return c

mediante_sql()

,0,1,2,3
0,68.0,MASCULINO,SUPE,Linaje B.1.621
1,66.0,MASCULINO,BARRANCA,Linaje C.37
2,33.0,MASCULINO,BARRANCA,Linaje B.1.617.2
3,35.0,FEMENINO,BARRANCA,Linaje P.1
4,31.0,MASCULINO,BARRANCA,Linaje B.1.617.2
5,25.0,MASCULINO,BARRANCA,Linaje B.1.617.2
6,42.0,MASCULINO,BARRANCA,Linaje C.37
7,14.0,FEMENINO,BARRANCA,Linaje AY.12
8,37.0,FEMENINO,BARRANCA,Sublinaje C.37


In [ ]:
def mediante_sql():

  cursor=conexión_a_variantes_covid_perú.cursor()

  comando_sql = '''
                SELECT Edad,Sexo, DISTRITO_PACIENTE, RESULTADO
                FROM  variantes_reportadas_en_perú
                WHERE PROVINCIA_PACIENTE = 'BARRANCA';
                '''
  # Ejecutar comado SQL con nuestro cursor
  for row in cursor.execute(comando_sql) :
    print(row)

mediante_sql()


(68.0, 'MASCULINO', 'SUPE', 'Linaje B.1.621 ')
(66.0, 'MASCULINO', 'BARRANCA', 'Linaje C.37 ')
(33.0, 'MASCULINO', 'BARRANCA', 'Linaje B.1.617.2 ')
(35.0, 'FEMENINO', 'BARRANCA', 'Linaje P.1 ')
(31.0, 'MASCULINO', 'BARRANCA', 'Linaje B.1.617.2 ')
(25.0, 'MASCULINO', 'BARRANCA', 'Linaje B.1.617.2 ')
(42.0, 'MASCULINO', 'BARRANCA', 'Linaje C.37 ')
(14.0, 'FEMENINO', 'BARRANCA', 'Linaje AY.12 ')
(37.0, 'FEMENINO', 'BARRANCA', 'Sublinaje C.37 ')


##**02.4. GUARDAMOS LOS CAMBIOS REALIZADOS EN LA BASE DE DATOS**

Para guardar los cambios realizados, o como punto de salvaguarda se usa la función  `conexión.commit()` ; y para cerrar la base de datos y permitir a la computadora operar sin retrasos o delays, cerramos la conexión  `conexión.close()` . \\
Los cambios efectuados como la creación y modificación de tablas serán guardados en la base de datos para futuras consultas.

In [ ]:
#Guardamos los cambiamos realizados
'''
La función conexión.commit() sirve como un punto de guardado de respaldo mientras
tenemos la base de datos abierta y modificamos las tablas y objetos contenidas.

Esto nos permite ante un error, regresar al último estado de commit guardado, siempre
que no hayamos cerrado la base de datos, mediante el comando:
conexión.rollback()
'''
conexión_a_variantes_covid_perú.commit()

#regresar al último estado commit guardado
conexión_a_variantes_covid_perú.rollback()

#Cerramos la base de datos y guardamos su estado actual
'''
Una vez cerrada la base de datos, todos los commits y objetos temporales son eliminados,
y no se puede volver a acceder a ellos, a menos que los volvamos a crear.
'''
conexión_a_variantes_covid_perú.close()


#**03. Ejemplos de Queries en SQL.**
-----------------------------
La herramienta más usada en SQL para el análisis de datos es la extracción mediante queries.
Usando Pandas y las conexiones sqlite3, son la forma más eficiente de extraer de archivos tipo .sql o .db .



```
SELECT selecciona las columnas
FROM es el nombre de la tabla específica
WHERE delimita las filas o condiciones
```



##**03.1. DEFINIMOS FUNCIÓN CON VALORES PREESTABLECIDOS Y MODIFICABLES.**
Definiré una función de dos parámetrs, la conexión que para estos ejemplos será el mismo : **VARIANTES_COVID_PERÚ.db** ; y el **query** que irá cambiando , pero tendrá como valor predeterminado en caso que no lo defina :


```
'SELECT *  FROM `variantes_reportadas_en_perú`'
```
Este query selecciona con * todas las columnas , por lo que devolverá todo el dataframe.





In [ ]:

def preguntando_a_base_de_datos_variantes_covid_perú( query ='SELECT *  FROM `variantes_reportadas_en_perú`;', base_de_datos = 'VARIANTES_COVID_PERÚ.db'):
  #Abrimos la conexión con la base de datos
  conexión = sqlite3.connect(base_de_datos)

  #Creamos el dataframe que será el obtenido al hacer la pregunta (QUERY) a la base de datos (CONEXIÓN)
  dataframe=pd.read_sql(query, conexión)

  #Cerramos la conexión establecida
  conexión.close()

  #La función nos retornará el objeto dataframe creado
  return dataframe

# Ejecuto la función sin definir algún parámetro, por lo que ejecutara los valores predeterminados
preguntando_a_base_de_datos_variantes_covid_perú()


,FECHA_CORTE,UUID,FECHA_MUESTRA,Edad,Sexo,Institucion,UBIGEO_PACIENTE,DEPARTAMENTO_PACIENTE,PROVINCIA_PACIENTE,DISTRITO_PACIENTE,DEPARTAMENTO_MUESTRA,PROVINCIA_MUESTRA,DISTRITO_MUESTRA,TIPO_MUESTRA,RESULTADO
0,2021-09-25 00:00:00,NaN,2021-02-09 00:00:00,93.0,FEMENINO,GOBIERNO REGIONAL,190115.0,PIURA,PIURA,VEINTISEIS DE OCTUBRE,PIURA,PIURA,PIURA,HISOPADO NASAL Y FARINGEO,Linaje B.1.1
1,2021-09-25 00:00:00,NaN,2021-02-22 00:00:00,31.0,FEMENINO,GOBIERNO REGIONAL,120101.0,LA LIBERTAD,TRUJILLO,TRUJILLO,LA LIBERTAD,TRUJILLO,TRUJILLO,HISOPADO NASAL Y FARINGEO,Linaje C.37
2,2021-09-25 00:00:00,NaN,2021-03-01 00:00:00,27.0,MASCULINO,GOBIERNO REGIONAL,120103.0,LA LIBERTAD,TRUJILLO,LAREDO,LA LIBERTAD,TRUJILLO,LAREDO,HISOPADO NASAL Y FARINGEO,Linaje C.37
3,2021-09-25 00:00:00,NaN,2021-03-07 00:00:00,34.0,MASCULINO,GOBIERNO REGIONAL,240102.0,,CALLAO,BELLAVISTA,CALLAO,PROV. CONST. DEL CALLAO,BELLAVISTA,HISOPADO NASAL Y FARINGEO,Linaje C.37
4,2021-09-25 00:00:00,NaN,2021-03-07 00:00:00,54.0,MASCULINO,GOBIERNO REGIONAL,240102.0,,CALLAO,BELLAVISTA,CALLAO,PROV. CONST. DEL CALLAO,BELLAVISTA,HISOPADO NASAL Y FARINGEO,Linaje C.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6366,2021-09-25 00:00:00,37259371.0,2021-08-18 00:00:00,36.0,FEMENINO,GOBIERNO REGIONAL,40109.0,AREQUIPA,AREQUIPA,PAUCARPATA,AREQUIPA,AREQUIPA,JACOBO HUNTER,HISOPADO NASAL Y FARINGEO,Linaje B.1.617.2
6367,2021-09-25 00:00:00,37269802.0,2021-08-01 00:00:00,51.0,MASCULINO,GOBIERNO REGIONAL,240101.0,,CALLAO,CALLAO,CALLAO,PROV. CONST. DEL CALLAO,CALLAO,HISOPADO NASAL Y FARINGEO,Linaje AY.4
6368,2021-09-25 00:00:00,37274411.0,2021-01-03 00:00:00,42.0,MASCULINO,GOBIERNO REGIONAL,140106.0,LIMA,LIMA,COMAS,CALLAO,PROV. CONST. DEL CALLAO,CALLAO,HISOPADO NASAL Y FARINGEO,Linaje B.1.2
6369,2021-09-25 00:00:00,37283811.0,2021-06-10 00:00:00,24.0,FEMENINO,GOBIERNO REGIONAL,40107.0,AREQUIPA,AREQUIPA,MIRAFLORES,AREQUIPA,AREQUIPA,MIRAFLORES,HISOPADO NASAL Y FARINGEO,Linaje P.1


##**03.2. GROUP BY , HAVING, COUNT, ORDER BY.**

Group by sirve para agrupar y having se utilliza como su delimitador. \\
La función COUNT(*) sirve par contar todas las filas, en este caso al agruparlas.

```
SELECT columna_1, COUNT(*) AS nuevo_nombre_asignado
FROM   seleccionar archivo
GROUP BY columna_1
HAVING delimita lo que se va a mostrar
ORDER BY columna_base_para_ordenar
```
A ORDER BY ,se le puede agregar DESC al final , para generar un orden descendente de mayor a menor.


In [ ]:
#"HALLAR LOS LINAJES DE COVID CON MÁS DE 100 REPORTES HASTA LA FECHA"
pregunta = '''
            SELECT RESULTADO AS Linaje_covid, COUNT(*) AS Número_de_reportes
            FROM   `variantes_reportadas_en_perú`
            GROUP BY Linaje_covid
            HAVING Número_de_reportes > 100
            ORDER BY Número_de_reportes DESC ;
            '''

# Ejecuto función con nuevo query
preguntando_a_base_de_datos_variantes_covid_perú(pregunta)

,Linaje_covid,Número_de_reportes
0,Linaje C.37,3564
1,Linaje P.1,1219
2,Linaje B.1.617.2,376
3,Linaje P.1.4,244
4,Linaje B.1.621,167
5,Linaje AY.12,116
6,Linaje AY.4,103


#**FINALMENTE, AUNQUE sqlite3 ES MUY VERSÁTIL Y NOS PERMITE ACCEDER A BASE DE DATOS Y ARCHIVOS TABLA, LO RECOMENDABLE ES USARLO PARA EXTRAER UNA TABLA QUE VAYA A SER MODIFICADA O FILTRADA DE FORMA MÁS ESPECÍFICA CON PANDAS, YA QUE EXISTEN FUNCIONES Y COMANDOS AVANZADOS DE SQL QUE NO SE PUEDEN EJECUTAR CON ESTA VERSIÓN LITE.**
